# Tutorial on online bin packing problem
## Please open in Colab !!!
Five steps:
1. Implement a sampler
2. Implement an evaluator and prepare a template program
3. Choose a method and run.

## Preparation: download the project file from GitHub. And update system path.

Download code from GitHub.

In [ ]:
!rm -rf llm4ad
!git clone https://github.com/Optima-CityU/llm4ad.git

In [2]:
from __future__ import annotations

import sys

sys.path.append('/content/llm4ad/')

from typing import Any, List

import llm4ad
from llm4ad.tools.profiler import TensorboardProfiler
from llm4ad.task.optimization.online_bin_packing import OBPEvaluation
from llm4ad.tools.llm.llm_api_https import HttpsApi

## 1. Implement an LLM interface
The sampler defines the way to use local LLM or LLM API. You should create a new Sampler class by implementing `llm4ad.base.LLM`.
- You should implement "draw_sample" function, to let the package know how to get a LLM's response by given a prompt.
- If you want more acceleration (such as batch inference, multi-threading sampling) you can also override "draw_samples" function.
- The following example shows a fake sampler, which returns a random function in the database.

The following example shows a sampler that uses API. If you want to use this sampler in this notebook, please complete following two variables.

In [9]:
api_endpoint: str = 'api.bltcy.ai'  # the ip of your API provider, no "https://", such as "api.bltcy.top".
api_key: str = ''  # your API key which may start with "sk-......"
model: str = 'gpt-4o-mini'

Use our `HttpsApi` class to access to your API provider.

In [10]:
sampler = llm4ad.tools.llm.llm_api_https.HttpsApi(host=api_endpoint, key=api_key, model=model)

In [11]:
# test the sampler
response = sampler.draw_sample('Hello!')
response

'Hello! How can I assist you today?'

## 2. Implement an evaluator and prepare a template program

In [12]:
import numpy as np


def get_valid_bin_indices(item: float, bins: np.ndarray) -> np.ndarray:
    """Returns indices of bins in which item can fit."""
    return np.nonzero((bins - item) >= 0)[0]


def online_binpack(
        items: tuple[float, ...], bins: np.ndarray, priority: callable
) -> tuple[list[list[float, ...], ...], np.ndarray]:
    """Performs online binpacking of `items` into `bins`."""
    # Track which items are added to each bin.
    packing = [[] for _ in bins]
    # Add items to bins.
    for item in items:
        # Extract bins that have sufficient space to fit item.
        valid_bin_indices = get_valid_bin_indices(item, bins)
        # Score each bin based on heuristic.
        priorities = priority(item, bins[valid_bin_indices])
        # Add item to bin with highest priority.
        best_bin = valid_bin_indices[np.argmax(priorities)]
        bins[best_bin] -= item
        packing[best_bin].append(item)
    # Remove unused bins from packing.
    packing = [bin_items for bin_items in packing if bin_items]
    return packing, bins


def evaluate(instances: dict, priority: callable) -> float:
    """Evaluate heuristic function on a set of online binpacking instances."""
    # List storing number of bins used for each instance.
    num_bins = []
    # Perform online binpacking for each instance.
    for name in instances:
        instance = instances[name]
        capacity = instance['capacity']
        items = instance['items']
        # Create num_items bins so there will always be space for all items,
        # regardless of packing order. Array has shape (num_items,).
        bins = np.array([capacity for _ in range(instance['num_items'])])
        # Pack items into bins and return remaining capacity in bins_packed, which
        # has shape (num_items,).
        _, bins_packed = online_binpack(items, bins, priority)
        # If remaining capacity in a bin is equal to initial capacity, then it is
        # unused. Count number of used bins.
        num_bins.append((bins_packed != capacity).sum())
    # Score of heuristic function is negative of average number of bins used
    # across instances (as we want to minimize number of bins).
    return -np.mean(num_bins)

In [13]:
template_program = '''
import numpy as np

def priority(item: float, bins: np.ndarray) -> np.ndarray:
    """Returns priority with which we want to add item to each bin.
    Args:
        item: Size of item to be added to the bin.
        bins: Array of capacities for each bin.
    Return:
        Array of same size as bins with priority score of each bin.
    """
    return bins - item
'''

The evaluator defines how to evaluate the generated heuristic function. You should create a new Evaluator class by implementing `llm4ad.base.Evaluation`. You should override "evaluate_program" function to specify. Return None if the function is invalid.

The `llm4ad.base.Evaluation` class provide acceleration and safe evaluation methods. You can use them by simply setting respective arguments. The commonly used two argument are:
- `use_numba_accelerate`: If set to True, we will wrap the heuristic function with '@numba.jit(nopython=True)'. Please note that not all functions support numba.jit(), so use it appropriately.
- `timeout_second`: Terminate the evaluation after timeout seconds. If set to `None`, the evaluator will wait until the evaluation finish.

For more arguments, please refer to docstring in `llm4ad.base.Evaluation`.

In [14]:
import pickle


class OBPEvaluator(llm4ad.base.Evaluation):
    """Evaluator for online bin packing problem."""

    def __init__(self):
        super().__init__(
            use_numba_accelerate=True,
            timeout_seconds=10,
            template_program=template_program,
        )
        try:
            with open('../online_bin_packing_fake/_data/weibull_train.pkl', 'rb') as f:
                self._bin_packing_or_train = pickle.load(f)['weibull_5k_train']
        except:
            with open('/content/llm4ad/example/online_bin_packing_fake/_data/weibull_train.pkl', 'rb') as f:
                self._bin_packing_or_train = pickle.load(f)['weibull_5k_train']

    def evaluate_program(self, program_str: str, callable_func: callable) -> Any | None:
        """Evaluate a given function. You can use compiled function (function_callable),
        as well as the original function strings for evaluation.
        Args:
            program_str: The function in string. You can ignore this argument when implementation.
            callable_func: The callable Python function of your sampled heuristic function code. 
            You can call the program using 'program_callable(args..., kwargs...)'
        Return:
            Returns the fitness value. Return None if you think the result is invalid.
        """
        # we call the _obp_evaluate.evaluate function to evaluate the callable code
        return evaluate(self._bin_packing_or_train, callable_func)

In [15]:
evaluator = OBPEvaluator()
secure_evaluator = llm4ad.base.SecureEvaluator(evaluator=evaluator, debug_mode=True)

### Test our evaluator

In [16]:
# test the evaluator
test_program = '''
import numpy as np

def priority(item: float, bins: np.ndarray) -> np.ndarray:
    """Returns priority with which we want to add item to each bin.
    Args:
        item: Size of item to be added to the bin.
        bins: Array of capacities for each bin.
    Return:
        Array of same size as bins with priority score of each bin.
    """
    return bins - item
'''

res = secure_evaluator.evaluate_program(test_program)
print(res)

DEBUG: evaluated program:
import numba
import numpy as np

@numba.jit(nopython=True)
def priority(item: float, bins: np.ndarray) -> np.ndarray:
    """Returns priority with which we want to add item to each bin.
    Args:
        item: Size of item to be added to the bin.
        bins: Array of capacities for each bin.
    Return:
        Array of same size as bins with priority score of each bin.
    """
    return bins - item

-5000.0


### Terminating evaluation test
Our platform will automatically terminate evaluation after `timeout_seconds` to prevent endless loop in the code. This is achieved by a secure evaluator.

In [18]:
# we test an invalid program
test_program = '''
import numpy as np

def priority(item: float, bins: np.ndarray) -> np.ndarray:
    """Returns priority with which we want to add item to each bin.
    Args:
        item: Size of item to be added to the bin.
        bins: Array of capacities for each bin.
    Return:
        Array of same size as bins with priority score of each bin.
    """
    while True:
        pass
'''

res = secure_evaluator.evaluate_program(test_program)
print(res)

DEBUG: evaluated program:
import numba
import numpy as np

@numba.jit(nopython=True)
def priority(item: float, bins: np.ndarray) -> np.ndarray:
    """Returns priority with which we want to add item to each bin.
    Args:
        item: Size of item to be added to the bin.
        bins: Array of capacities for each bin.
    Return:
        Array of same size as bins with priority score of each bin.
    """
    while True:
        pass

DEBUG: the evaluation time exceeds 10s.
None


## 3. Choose a method and run
Our package support multiprocess running. However, the Colab backend has limited CPU support, so we set num_evlauators to 2.

In [19]:
from llm4ad.tools.profiler import ProfilerBase
from llm4ad.method.randsample import RandSample

# you can also try other LLM-EPS methods.
rand_sample = RandSample(
    llm=sampler,
    profiler=ProfilerBase(log_dir='logs/randomsample', log_style='simple'),
    evaluation=OBPEvaluation(),
    max_sample_nums=10,
    num_samplers=2,
    num_evaluators=2,
)

==================================LLM Parameters===============================
LLM: HttpsApi
do_auto_trim: True
debug_mode: False
_host: api.bltcy.ai
_key: sk-z4tSPqfaQ74KBpTlFc4f7fC767D04603Be0696F7A8BcC7D8
_model: gpt-4o-mini
_timeout: 20
_kwargs: {}
_cumulative_error: 0
==================================Problem Parameters===============================
Problem: OBPEvaluation
task_description: Implement a function that returns the priority with which we want to add an item to each bin.
use_numba_accelerate: False
use_protected_div: False
protected_div_delta: 1e-05
random_seed: None
timeout_seconds: 20
exec_code: True
safe_evaluate: True
daemon_eval_process: False
==================================Method Parameters===============================
Method: RandSample
_max_sample_nums: 10
_num_samplers: 1
_num_evaluators: 1
_debug_mode: False
_resume_mode: False
_function_to_evolve_name: priority
==================================End of Parameters===============================


In [20]:
if __name__ == '__main__':
    rand_sample.run()

Sample1: Score=-2091.800     Cur_Best_Score=-2091.800
Sample2: Score=None    Cur_Best_Score=-2091.800
Sample3: Score=-5000.000     Cur_Best_Score=-2091.800
Sample4: Score=-5000.000     Cur_Best_Score=-2091.800
Sample5: Score=-5000.000     Cur_Best_Score=-2091.800


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x110e329d0>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/alevo311/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 